## Load X_test and y_test

In [14]:
import pandas as pd
import os
import torch

X_test_path = os.path.join(os.getcwd(), '../', 'data', 'processed', 'X_test_wiht_outliers.csv')
y_test_path = os.path.join(os.getcwd(), '../', 'data', 'processed', 'y_test_wiht_outliers.csv')

X_test = pd.read_csv(X_test_path)
y_test = pd.read_csv(y_test_path)

X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

## Load model

In [15]:
import os
import sys
import torch

# Get the project root
project_root = os.path.abspath("..")    # Go up one level from "notebooks/"
sys.path.append(project_root)           # Add the root to the path

from src.models.train_model import *
from src.tests.test_models import *

In [16]:
best_model_path = os.path.join(os.getcwd(), '../', 'src', 'models', 'avg_with_outliers.pth')

loaded_model = RegressionModel(5)
loaded_model.load_state_dict(torch.load(best_model_path, weights_only=True))
criterion = RMSELoss()

## Evaluate model

In [17]:
# Evaluación final en el conjunto de prueba
loaded_model.eval()
with torch.no_grad():
    test_outputs = loaded_model(X_test)
    test_loss = criterion(test_outputs, y_test)
    print(f'Final Test Loss (RMSE): {test_loss.item():.4f}')

Final Test Loss (RMSE): 0.8661


## Metrics

In [18]:
import torch

loaded_model.eval()                # Set the model to evaluation mode
with torch.no_grad():
    # Predictions for test set
    test_outputs = loaded_model(X_test)
    
    # Get metrics
    mse, rmse, mae, mape = calculate_metrics(y_test, test_outputs)

print(f"Test Metrics:")
print(f"\tMean Squared Error (MSE): {mse:.4f}")
print(f"\tRoot Mean Squared Error (RMSE): {rmse:.4f}")
print(f"\tMean Absolute Error (MAE): {mae:.4f}")
print(f"\tMean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(f"\tAcurracy: {100 - mape:.2f}%")

Test Metrics:
	Mean Squared Error (MSE): 0.5390
	Root Mean Squared Error (RMSE): 0.7342
	Mean Absolute Error (MAE): 0.5593
	Mean Absolute Percentage Error (MAPE): 7.38%
	Acurracy: 92.62%


## Testing

In [19]:
# Data for prediction
manual_input = preprocess_input(
    escuela='ESCOM',
    materias_reprobadas=0,
    apoyo_familiar=3,
    apoyos_economicos='Sí',
    ejercer_carrera='Sí'
)

manual_input = manual_input.unsqueeze(0)

# model.eval()
loaded_model.eval()

# Make prediction
with torch.no_grad():
    prediction = loaded_model(manual_input)

# Show prediction
print(f'Predicted Promedio Global: {prediction.item():.2f}')

Predicted Promedio Global: 8.24
